In [340]:
import pandas as pd
from pandas import DataFrame, read_csv, Series
import numpy as np
import math
import sklearn.datasets
from sklearn.cluster import KMeans
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [341]:
# получение данных о студентах и их оценках по дисциплинам
import csv
def read_file(filename):
    f = open(filename)
    data = csv.reader(f)
    mentions = dict()
    for line in data:
        student = line[0]
        discipline = line[6]
        type_d = line[7]
        sem_rate = int(line[8])
        bonus = int(line[9])
        exam_rate = int(line[10])
        rate = 0
        if type_d == 'exam':
            if (sem_rate >= 38) & (exam_rate >= 22):
                rate = sem_rate + exam_rate + bonus
        else: 
            if (sem_rate >= 60):
                rate = sem_rate + bonus
        if not student in mentions:
            mentions[student] = dict()
        mentions[student][discipline] = rate
    f.close()
    return mentions

In [342]:
# косинусная мера сходства: насколько два пользователя похожи друг на друга. 
def dist_cosine(vecA, vecB):
    def dot_product(vecA, vecB):
        d = 0.0
        for dim in vecA:
            if dim in vecB: 
                d += vecA[dim]*vecB[dim]
        return d
    a = dot_product(vecA,vecB)
    b = math.sqrt(dot_product(vecA,vecA))
    c = math.sqrt(dot_product(vecB,vecB))
    if (a == 0) or (b == 0) or (c == 0):
        return 0
    else:
        return  (a / (b * c))

In [343]:
def similar_students(studentID, studentsData, nBestStudents = 3):
    matches = [(u, dist_cosine(studentsData[studentID], studentsData[u])) for u in studentsData.keys() if u != studentID]
    bestMatches = sorted(matches, key=lambda q:(q[1], q[0]), reverse=True)[:nBestStudents]
    print("Most correlated with studentID: " + str(studentID))
    for line in bestMatches:
        print("StudentID: " + str(line[0]) + "  Coeff: " + str(line[1])) 
    return [(x[0], x[1]) for x in bestMatches]

In [344]:
students_test = read_file("raw_data32.csv")
#df = DataFrame(students_test)
#df.columns.name = 'studentID'
#df.index.name = 'discipline'    
rec = similar_students('1', students_test, 10)


Most correlated with studentID: 1
StudentID: 9  Coeff: 0.7542152087381664
StudentID: 6  Coeff: 0.7248624347806895
StudentID: 7  Coeff: 0.7194365076928019
StudentID: 5  Coeff: 0.6837763094049188
StudentID: 4  Coeff: 0.5597650673256156
StudentID: 2  Coeff: 0.22271363971105235
StudentID: 3  Coeff: 0.06705017139158977
StudentID: 8  Coeff: 0


In [345]:
# Похожесть дисциплин
def read_subjects(filename):
    f = open(filename)
    data = csv.reader(f)
    subjects = dict()
    for line in data:
        ID = line[0]
        discipline = line[1]
        subjects[ID] = discipline
    f.close()
    return subjects

In [346]:
#sd = read_subjects("subjects.csv")

#nltk.download() 

stopwords = set(stopwords.words("russian"))
#print(stopwords)


In [347]:
def review_to_words(review_text):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return(" ".join(meaningful_words))

In [348]:
d0 = pd.read_csv("subjects2.csv", encoding='utf-8')
d0.head(3)

,ID,Name,Abbr
0,1,Математическая логика и теория множеств,Мат. логика и ТМ
1,2,Математический анализ,Мат. Анализ
2,3,Алгебра и геометрия,Алг. и геом.


In [349]:
d1 = d0.copy()
d1['Name'] = d1['Name'].apply(lambda s: review_to_words(s))
d1.head(3)

,ID,Name,Abbr
0,1,математическая логика теория множеств,Мат. логика и ТМ
1,2,математический анализ,Мат. Анализ
2,3,алгебра геометрия,Алг. и геом.


In [350]:
#vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 50000, binary = False) 
#vectorized = vectorizer.fit_transform(d1['Name'])
##print(vectorized)

#num_samples, num_features = vectorized.shape
## print((num_samples, num_features)) # 918 дисциплин, 1123-мерный вектор признаков

#vocabulary = vectorizer.get_feature_names()
##print(vocabulary[20])

#num_clusters = 50
#km = KMeans(n_clusters=num_clusters, init='random', n_init=10, verbose=1, random_state=3)
#km.fit(vectorized)


In [351]:
#km.predict(vectorized)
#km.cluster_centers_

In [352]:
##print(km.cluster_centers_)
#print(km.labels_)

In [353]:
#d1['Name'][km.labels_ == 46]

In [354]:
# стемминг
from nltk.stem.snowball import SnowballStemmer
# print(SnowballStemmer("russian").stem("гениально"))  # проверка работы стемминга

In [355]:
rus_stemmer = SnowballStemmer('russian')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (rus_stemmer.stem(w) for w in analyzer(doc))
    
vectorizer = StemmedCountVectorizer(min_df=1, analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = None, binary = False)
vectorized = vectorizer.fit_transform(d1['Name'])

In [356]:
num_samples, num_features = vectorized.shape
vocabulary = vectorizer.get_feature_names()

num_clusters = 650
km = KMeans(n_clusters=num_clusters, init='k-means++', n_init=8, verbose=1, random_state=3)
km.fit(vectorized)

Initialization complete
Iteration  0, inertia 241.000
Iteration  1, inertia 182.033
Converged at iteration 1: center shift 0.000000e+00 within tolerance 4.368989e-07
Initialization complete
Iteration  0, inertia 245.000
Iteration  1, inertia 190.714
Converged at iteration 1: center shift 0.000000e+00 within tolerance 4.368989e-07
Initialization complete
Iteration  0, inertia 242.000
Iteration  1, inertia 186.224
Iteration  2, inertia 185.805
Iteration  3, inertia 185.461
Converged at iteration 3: center shift 0.000000e+00 within tolerance 4.368989e-07
Initialization complete
Iteration  0, inertia 249.000
Iteration  1, inertia 192.451
Converged at iteration 1: center shift 0.000000e+00 within tolerance 4.368989e-07
Initialization complete
Iteration  0, inertia 243.000
Iteration  1, inertia 190.427
Converged at iteration 1: center shift 0.000000e+00 within tolerance 4.368989e-07
Initialization complete
Iteration  0, inertia 244.000
Iteration  1, inertia 185.786
Iteration  2, inertia 184.

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=650, n_init=8, n_jobs=1, precompute_distances='auto',
    random_state=3, tol=0.0001, verbose=1)

In [357]:
km.predict(vectorized)
km.cluster_centers_

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [358]:
print(km.labels_)

[471 466  39 582 266  46  11  32 490 533 456 343  11  39  72  11  11 597
 390  11  37  11  23 411  25 130  96  71 617  37 157 369   4  60 394  67
 636  41  64 359  12 587 547  11 446 108   7 582 180 262  46 283 224 154
 266  11  11 470 586 538 541 616 347 111 537  12  12 323 325  15 287   1
  11 589  11 378 166 576  59 464 266 534 584  11 266 645 151 429 584 164
 181 325 327 229  12  81 167 445 458  99  15 507 421 638 452   1  46  11
  15 509 130 608 385 409 428  11  39 596  65 297 627 354 528 460 295  87
  19 102  79  33 530 381 282  53  20 277  78 235 226 417 208 501 544 571
 484 109 539 236  25  88 621 556 114 273 118 348  22  24 257 205  11 518
 351  26  72 531  60 279 194 106  58  35 209 177  43 320 249 346  38 505
 633   2   2 482 418 485 364  48 328  11 603 175 383  11  11  11 549 193
 492 186 135 294 232 566 231 171 611 188  48 266 392 410 468 419 418 173
 598 101 228 483 187  13 461 142  50 331   4 512 433 172 200 336  11  32
  94 337  11 516  36  35  79 263  33  61 100 573 51

In [359]:
d1['Name'][km.labels_ == 11]
    # 25 кластер "очень плохой" при init = random, num_kluster = 500, n_init = 13, неправильно соотносит предмет "Теория графов"
    # 41 кластер "плохой" при init = k_means++, num_kluster = 500, n_init = 13
    # 62 кластер "плохой" при init = k_means++, num_kluster = 523, n_init = 13
    # хорошое разбиение при init = k_means++, num_kluster = 700, n_init = 13, но очень много "одиночных" дисциплин
    # 23 кластер "плохой" при init = k_means++, num_kluster = 600, n_init = 13
    # 79 кластер "плохой" при init = k_means++, num_kluster = 600, n_init = 15

6                     консультация
12                   физвоспитание
15                    русский язык
16          самостоятельная работа
19                  спецподготовка
21                    введение ооп
43              комплексный анализ
55                    правоведение
56                             бжд
72                             ксе
74                     факультатив
83                             нис
107          микропрограммирование
115                               
160                     социология
189            гражданский процесс
193                 криминалистика
194         профессиональная этика
195                   криминология
232                   аэроакустика
236                 гидроупругость
288                            ооп
296                           субд
316                               
319              нелинейные модели
320                               
325                               
340                               
341                 

In [360]:
import scipy as sp

def disciplines_in_one_cluster(from_info, discipline):
    discipline = review_to_words(discipline)
    discipline_vec = vectorizer.transform([discipline])
    discipline_label = km.predict(discipline_vec)[0]
    return from_info[km.labels_ == discipline_label]

In [361]:
print("Test 1")
sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Теория графов")
print(sim_dis_info)
print()

print("Test 2")
sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Компьютерные сети")
print(sim_dis_info)
print()

print("Test 3")
sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Механика")
print(sim_dis_info)
print()

print("Test 4")
sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Алгоритмы на графах")
print(sim_dis_info)

Test 1
659    теория конечных графов
Name: Name, dtype: object

Test 2
37                   компьютерные сети
736    практикум эвм компьютерные сети
840        компьютерные сети практикум
Name: Name, dtype: object

Test 3
104    механика материалов
Name: Name, dtype: object

Test 4
49                 алгоритмы графах
491    алгоритмы оптимизации графах
Name: Name, dtype: object


In [362]:
# рекомендации для студента id_student
# информацию о студентах получаем из students_info
# выбираем предметы, основываясь на схожести с n_best_students наиболее похожими студентами
def make_recomendations(id_student, students_info, subjects_info, n_best_students = 3):
    sim_stud = similar_students(id_student, students_info, n_best_students)  
    stud_disciplines = [] 
    disciplines_info = read_subjects("subjects.csv")
    for d in students_info[id_student].keys():
        if d not in stud_disciplines:
            stud_disciplines.append(d)
            #stud_disciplines.append(disciplines_info[d])    # дисциплины студента id_student
            #sim_dis_info = disciplines_in_one_cluster(subjects_info, disciplines_info[d])
            #for sim_dis in sim_dis_info:
                #if sim_dis not in stud_disciplines:
                    #stud_disciplines.append(sim_dis) # добавление дисциплин из одного кластера. Проблема : обрезанные/исправленные имена дисциплин
    # далее добавление дисциплин "похожих студентов" + дисциплины из одних с ним кластеров
    # далее убираем все дисциплины, принадлежащие одному кластеру, если есть у id_stud плохие оценки по предметам из этого кластера
    # тут возникает новая проблема : а вдруг по предметам из одного кластера у студента разные оценки, например : матан1 - 60 баллов, матан2 - 90 баллов
    return stud_disciplines

In [363]:
students_rating = read_file("students_rating.csv")
r = make_recomendations('11', students_rating, d1['Name'])
r

Most correlated with studentID: 11
StudentID: 5  Coeff: 0.9729210307385605
StudentID: 18  Coeff: 0.9683713392884907
StudentID: 15  Coeff: 0.967386392071021


['11',
 '31',
 '49',
 '52',
 '163',
 '65',
 '424',
 '491',
 '569',
 '579',
 '668',
 '694',
 '700',
 '802',
 '1009',
 '1054',
 '1058',
 '1154',
 '1176',
 '1200',
 '1204',
 '1233',
 '1253',
 '1475',
 '1486',
 '1506',
 '1514',
 '1631',
 '1651',
 '1686',
 '1725',
 '1728',
 '1830',
 '1870',
 '1892',
 '1897',
 '1918',
 '1981',
 '2014',
 '2033',
 '2075',
 '2103',
 '2137',
 '2156',
 '2245',
 '2322',
 '2341',
 '2346',
 '2373']